In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

sns.set()

%matplotlib inline
%precision 3

'%.3f'

In [3]:
2^6

4

In [35]:
train = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/train.csv',index_col=0)
test = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/test.csv',index_col=0)
sample_submission = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/sample_submission.csv',index_col=0)

In [36]:
label_encoder = LabelEncoder()

train.type = label_encoder.fit_transform(train.type)

res_train_y = train.type
res_train_x = train.iloc[:,1:]

res_test_x = test

In [37]:
train_x,test_x,train_y,test_y = train_test_split(train.iloc[:,1:],train.type,random_state=2)

In [38]:
############################################################################
############ Random Forest
############################################################################
estimators = 100
np.random.seed(2)
RF_model = RandomForestClassifier(n_estimators = estimators)
RF_model.fit(train_x, train_y)
RF_prediction = RF_model.predict_proba(test_x)

In [40]:
# 100
print("log_loss : ",log_loss(test_y,RF_prediction))

log_loss :  0.4781892666217565


In [ ]:
submission_RF = pd.DataFrame(data=RF_prediction, columns=sample_submission.columns, index=test_x.index)
submission_RF.to_csv('submission_RF.csv', index=True)

In [ ]:
############################################################################
############ Ada Boosting
############################################################################
estimator = DecisionTreeClassifier(max_depth=3,random_state=2,criterion='entropy')
ADA_model = AdaBoostClassifier(base_estimator=estimator,
                               n_estimators=500,
                               random_state=2,
                               learning_rate=0.1
                              )

ADA_model.fit(train_x,train_y)
ADA_prediction = ADA_model.predict_proba(x_test)

In [ ]:
print("log_loss : ",log_loss(test_y,ADA_prediction))

In [ ]:
############################################################################
############ GBM
############################################################################
estimators = 50
np.random.seed(2)
GBM_model = GradientBoostingClassifier(n_estimators = estimators)
GBM_model.fit(train_x, train_y)
GBM_prediction = GBM_model.predict_proba(test_x)

In [ ]:
print("log loss : ",log_loss(test_y,GBM_prediction))

In [ ]:
submission_GBM = pd.DataFrame(data=GBM_prediction, columns=sample_submission.columns, index=test_x.index)
submission_GBM.to_csv('submission_GBM.csv', index=True)

In [53]:
############################################################################
############ XGBOOST
############################################################################
dtrain_prod = xgb.DMatrix(data = train_x, label = train_y)
dtest_prod = xgb.DMatrix(data = test_x,label=test_y)

# Custom error function for the XGB model
threshold = 0.5
def eval_error(preds, dtrain):
    labels = dtrain.get_label()
    preds = (preds > threshold ).astype('float')
    return "accuracy", accuracy_score(labels, preds)

param = {'objective' : 'multi:softprob',
         'max_depth' : 9,
         'eta': 0.3,
         'num_class': 19
#          'colsample_bytree' : 1,
#          'subsample' : 1
         }


nrounds = 2

np.random.seed(2)
XGB_model = xgb.train(param, 
                      dtrain_prod, 
                      num_boost_round = nrounds ,
                      feval = eval_error,
                      maximize = True,
#                       early_stopping_rounds = 10,
                      )

XGB_prediction = XGB_model.predict(dtest_prod)

C:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [54]:
print("log loss : ",log_loss(test_y,XGB_prediction))

log loss :  0.953463232917742


In [ ]:
submission_XGB = pd.DataFrame(data=XGB_prediction, columns=sample_submission.columns, index=test_x.index)
submission_XGB.to_csv('submission_XGB.csv', index=True)

In [ ]:
############################################################################
#Ensembling the three models
############################################################################

#Forming the ensemble dataset of the 3 models
# ensemble = pd.DataFrame()
# ensemble = (submission_XGB + submission_GBM + submission_RF)/3

# #Printing to see all the hospitals that are classified as closed 
# print(ensemble.loc[ensemble['OC'] == 0, ])

# ensemble = ensemble.loc[:, ['inst_id', 'OC']]

# ensemble.to_csv('ens.csv')

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(train_x, train_y)